In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import random
import os

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Print current working directory to understand file paths
print("Current working directory:", os.getcwd())

Current working directory: /home/omar/Downloads/SDR/Notebooks


In [5]:
# Define paths to the dataset files
normal_path = "../Dataset/training/Rssi_Normal.txt"
cj_path = "../Dataset/training/Rssi_CJ.txt"
pj_path = "../Dataset/training/Rssi_PJ.txt"

# Function to load the data
def load_data(file_path):
    try:
        data = np.loadtxt(file_path)
        return data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return np.array([])

# Load the datasets
normal_data = load_data(normal_path)
cj_data = load_data(cj_path)
pj_data = load_data(pj_path)

# Print the shapes to verify data loading
print(f"Normal data shape: {normal_data.shape}")
print(f"Constant Jammer data shape: {cj_data.shape}")
print(f"Pulsed Jammer data shape: {pj_data.shape}")

# Select random samples from each class (balanced)
samples_per_class = 200 // 3  # Approximately equal samples per class
remaining_samples = 200 - (samples_per_class * 3)  # Handle any remainder

# Select random indices for each class
normal_indices = np.random.choice(len(normal_data), samples_per_class, replace=False)
cj_indices = np.random.choice(len(cj_data), samples_per_class, replace=False)
pj_indices = np.random.choice(len(pj_data), samples_per_class + remaining_samples, replace=False)

# Extract the samples
normal_samples = normal_data[normal_indices]
cj_samples = cj_data[cj_indices]
pj_samples = pj_data[pj_indices]

# Create labels (0: Normal, 1: Constant Jammer, 2: Pulsed Jammer)
normal_labels = np.zeros(samples_per_class)
cj_labels = np.ones(samples_per_class)
pj_labels = np.full(samples_per_class + remaining_samples, 2)

# Combine samples and labels
X_test = np.concatenate([normal_samples, cj_samples, pj_samples])
y_test = np.concatenate([normal_labels, cj_labels, pj_labels])

# Reshape X_test to match model input requirements (assuming 1D input)
X_test = X_test.reshape(X_test.shape[0], 1)

print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"Class distribution: {np.bincount(y_test.astype(int))}")

Normal data shape: (695552,)
Constant Jammer data shape: (694278,)
Pulsed Jammer data shape: (694642,)
Test data shape: (200, 1)
Test labels shape: (200,)
Class distribution: [66 66 68]


In [ ]:
# Check the model's input shape requirements
def get_model_input_shape(model):
    """Extract the expected input shape from the model"""
    # Get the first layer
    first_layer = model.layers[0]
    # Get the input shape
    input_shape = first_layer.input_shape
    return input_shape

# This function will be used after loading the model

In [ ]:
# Load the saved model from the model directory
model_path = "../model/model_NewDataset.h5"

# Try to load the model with custom_objects parameter
model = load_model(model_path, compile=False)
print(f"Successfully loaded model from {model_path}")
# Display model summary
model.summary()

Error loading model: Unrecognized keyword arguments: ['batch_shape']

Trying alternative model...
Error loading alternative model: Unrecognized keyword arguments: ['batch_shape']

Please check the model files and format.


In [ ]:
# Check the model's expected input shape
input_shape = get_model_input_shape(model)
print(f"Model expects input shape: {input_shape}")

# Examine the model's first few layers to understand structure
for i, layer in enumerate(model.layers[:3]):
    print(f"Layer {i}: {layer.name}, Input shape: {layer.input_shape}, Output shape: {layer.output_shape}")

In [ ]:
# Reshape X_test to match model input requirements
# Check the dimensionality requirements
if input_shape[1] is not None:  # If the model expects a specific sequence length
    # Get sample dimensionality
    sample_dim = X_test.shape[1] if len(X_test.shape) > 1 else 1
    
    # If samples are 1D and model expects 2D or 3D input
    if len(input_shape) >= 3:
        # Reshape to (samples, sequence_length, features)
        # For Conv1D, we need at least 3 time steps for a kernel size of 3
        sequence_length = max(3, input_shape[1]) if input_shape[1] is not None else 3
        
        # If each sample is just a single value, we need to expand it
        if sample_dim == 1:
            print("Expanding single values to sequences...")
            # Create sequences by repeating each value
            X_test_expanded = np.repeat(X_test, sequence_length).reshape(-1, sequence_length)
            # Add feature dimension if needed
            if len(input_shape) == 3:  # Model expects (batch, seq_len, features)
                X_test = X_test_expanded.reshape(-1, sequence_length, 1)
            else:
                X_test = X_test_expanded
        else:
            # If samples already have multiple values, reshape appropriately
            if len(input_shape) == 3:  # Model expects (batch, seq_len, features)
                X_test = X_test.reshape(-1, X_test.shape[1], 1)
    
    # If the model just expects a flattened input
    elif len(input_shape) == 2:  # Model expects (batch, features)
        X_test = X_test.reshape(X_test.shape[0], -1)

print(f"Reshaped test data to: {X_test.shape}")

In [ ]:
# Make predictions on the test data
y_pred_probs = model.predict(X_test)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1) if y_pred_probs.shape[1] > 1 else np.round(y_pred_probs).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Display confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Display detailed classification report
print("\nClassification Report:")
target_names = ['Normal', 'Constant Jammer', 'Pulsed Jammer']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# Visualize the confusion matrix
plt.figure(figsize=(10, 8))

# Use a color map that makes it clear which values are higher
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.imshow(cm_normalized, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Normalized Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(target_names))
plt.xticks(tick_marks, target_names, rotation=45)
plt.yticks(tick_marks, target_names)

# Add text annotations to each cell
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, f"{cm[i, j]} ({cm_normalized[i, j]:.2f})",
                 ha="center", va="center",
                 color="white" if cm_normalized[i, j] > 0.5 else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Create a bar chart comparing true vs predicted class distribution
plt.figure(figsize=(10, 6))

true_counts = np.bincount(y_test.astype(int), minlength=3)
pred_counts = np.bincount(y_pred.astype(int), minlength=3)

x = np.arange(len(target_names))
width = 0.35

plt.bar(x - width/2, true_counts, width, label='True Labels')
plt.bar(x + width/2, pred_counts, width, label='Predicted Labels')

plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('True vs Predicted Class Distribution')
plt.xticks(x, target_names)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()